# READ MODIS

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import glob

In [3]:
test = xr.open_dataset('MODIS/PARMonthlyClimatology/A20021822019212.L3m_MC_PAR_par_9km.nc')

In [4]:
test

<xarray.Dataset>
Dimensions:  (eightbitcolor: 256, lat: 2160, lon: 4320, rgb: 3)
Coordinates:
  * lat      (lat) float32 89.958336 89.875 89.79167 ... -89.87501 -89.958336
  * lon      (lon) float32 -179.95833 -179.875 ... 179.87502 179.95836
Dimensions without coordinates: eightbitcolor, rgb
Data variables:
    par      (lat, lon) float32 ...
    palette  (rgb, eightbitcolor) uint8 ...
Attributes:
    product_name:                      A20021822019212.L3m_MC_PAR_par_9km.nc
    instrument:                        MODIS
    title:                             MODISA Level-3 Standard Mapped Image
    project:                           Ocean Biology Processing Group (NASA/G...
    platform:                          Aqua
    temporal_range:                    17-year
    processing_version:                2018.0
    date_created:                      2019-09-09T18:30:32.000Z
    history:                           l3mapgen par=A20021822019212.L3m_MC_PA...
    l2_flag_names:                   

In [2]:
TempNCDFfiles = glob.glob('MODIS/PARMonthlyClimatology/*.nc', recursive=True)
MODIS = xr.open_mfdataset(TempNCDFfiles,combine='by_coords',aggdim=)#,decode_times=False)

ValueError: Could not find any dimension coordinates to use to order the datasets for concatenation